# 👹 Fera Formidável 4.9

> Atividade realizada em dupla: Caio Ruas (24010) e Thalles Cansi (24006)

<!-- Resolver o problema da senha de forma que você não forneça a informação
do tamanho da senha para a função que gera a população. Considere que a senha pode
ser uma string de 1 até 30 caracteres.
Dica: A função objetivo terá que quantificar em sua métrica tanto se o candidato
acertou as letras quanto se acertou o tamanho da senha.
8
Dica 2: Você pode criar diferentes estratégias de mutação, não precisa ser apensa
uma! Quem sabe uma função de mutação pode alterar letras e a outra pode alterar o
tamanho da senha? Ver o exercício “Praticamente um X-man!”.
Dica 3: Observe que você terá que pensar um pouco sobre como fará o cruzamento no
caso de senhas de tamanhos diferentes. Quem sabe tenha que fazer alguma consideração
adicional sobre quais são os valores possíveis para o ponto de corte… -->

Mais uma fera formidável, mais uma atividade de programação funcional! Para esta fera, vamos ter que resolver o problema da senha de forma que você não forneça a informação do tamanho da senha para a função que gera a população. Vamos considerar que a senha pode ser uma string de 1 até 30 caracteres.

Vamos começar importando as bibliotecas que iremos usar. Neste caso, vamos usar `random` para gerar números aleatórios e `string` para obter os caracteres alfanuméricos que serão usados na senha.

In [1]:
import random
from string import ascii_lowercase, ascii_uppercase, digits, punctuation

random.seed(7)

Vamos também importar todas as funções que defininimos no arquivo `Scripts/FeraFormidável49.py`, que contém as funções necessárias para resolver o problema da senha com Algoritmos Genéticos.

In [2]:
from Scripts.FeraFormidavel49 import populacao_senha_var_len
from Scripts.FeraFormidavel49 import funcao_objetivo_pop_senha_var_len
from Scripts.FeraFormidavel49 import selecao_torneio_min
from Scripts.FeraFormidavel49 import cruzamento_uniforme_var_len
from Scripts.FeraFormidavel49 import mutacao_salto_var_len
from Scripts.FeraFormidavel49 import mutacao_simples_var_len
from Scripts.FeraFormidavel49 import mutacao_altera_tamanho

Para iniciarmos, vamos definir os hiperparâmetros do algoritmo genético e as constantes que serão usadas no código. Esses parâmetros incluem o tamanho da população, as chances de cruzamento e mutação, o tamanho do torneio, o tamanho mínimo e máximo da senha, e a penalidade por diferença de tamanho.

In [3]:
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits + punctuation
LISTA_CARACTERES_POSSIVEIS = list(CARACTERES_POSSIVEIS)

TAMANHO_POPULACAO = 100

CHANCE_DE_CRUZAMENTO = 0.7
CHANCE_DE_MUTACAO_CHAR = 0.05
CHANCE_DE_MUTACAO_TAMANHO = 0.15

TAMANHO_TORNEIO = 3

MIN_SENHA_LEN = 1
MAX_SENHA_LEN = 30

PENALTY_PER_LENGTH_DIFF = 75

Agora que já temos os hiperparâmetros definidos, vamos criar a população inicial de senhas. Para isso, vamos usar a função `populacao_senha_var_len`, que gera uma população de senhas com tamanhos variáveis entre o mínimo e o máximo definidos.

In [4]:
populacao = populacao_senha_var_len(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS, MIN_SENHA_LEN, MAX_SENHA_LEN)

menor_fitness_geral = float("inf")
geracao = 0
melhor_candidato_historico = []

Agora, vamos definir a nossa senha alvo. Para isso, vamos solicitar ao usuário que digite uma senha entre 1 e 30 caracteres. Vamos garantir que a senha não seja vazia e que esteja dentro dos limites definidos. Também vamos verificar se a senha contém apenas caracteres válidos.

In [5]:
while True:
    SENHA_ALVO = list(input("Digite sua senha entre 1 e 30 caracteres: ").strip())
    if not SENHA_ALVO:
        print("Senha não pode ser vazia.")
        continue
    if len(SENHA_ALVO) < MIN_SENHA_LEN or len(SENHA_ALVO) > MAX_SENHA_LEN:
        print(f"Senha deve ter entre {MIN_SENHA_LEN} e {MAX_SENHA_LEN} caracteres.")
        continue
    if any(c not in LISTA_CARACTERES_POSSIVEIS for c in SENHA_ALVO):
        print("Senha contém caracteres inválidos.")
        continue
    break

Excelente! Temos a senha alvo definida e a população inicial gerada. Agora, vamos implementar o loop principal do algoritmo genético, onde iremos iterar até que encontremos a senha correta.

In [6]:
while menor_fitness_geral != 0:
    fitness = funcao_objetivo_pop_senha_var_len(populacao, SENHA_ALVO, PENALTY_PER_LENGTH_DIFF)

    selecionados = selecao_torneio_min(populacao, fitness, TAMANHO_TORNEIO)

    proxima_geracao = []

    random.shuffle(selecionados)
    for i in range(0, len(selecionados) - 1, 2):
        pai = selecionados[i]
        mae = selecionados[i + 1]

        filho1, filho2 = cruzamento_uniforme_var_len(
            pai,
            mae,
            CHANCE_DE_CRUZAMENTO,
            CARACTERES_POSSIVEIS,
            MAX_SENHA_LEN,
        )
        proxima_geracao.append(filho1)
        proxima_geracao.append(filho2)

    if len(selecionados) % 2 == 1:
        proxima_geracao.append(list(selecionados[-1]))

    while len(proxima_geracao) < TAMANHO_POPULACAO:
        if proxima_geracao:
            proxima_geracao.append(list(random.choice(proxima_geracao)))
        else:
            proxima_geracao.append(
                cria_candidato_senha_var_len(
                    CARACTERES_POSSIVEIS,
                    MIN_SENHA_LEN,
                    MAX_SENHA_LEN,
                )
            )

    mutacao_simples_var_len(
        proxima_geracao,
        CHANCE_DE_MUTACAO_CHAR,
        LISTA_CARACTERES_POSSIVEIS,
    )

    mutacao_salto_var_len(
        proxima_geracao,
        CHANCE_DE_MUTACAO_CHAR,
        LISTA_CARACTERES_POSSIVEIS,
    )

    mutacao_altera_tamanho(
        proxima_geracao,
        CHANCE_DE_MUTACAO_TAMANHO,
        CARACTERES_POSSIVEIS,
        MIN_SENHA_LEN,
        MAX_SENHA_LEN,
    )

    populacao = proxima_geracao
    geracao += 1

    fitness_atualizado = funcao_objetivo_pop_senha_var_len(
        populacao, SENHA_ALVO, PENALTY_PER_LENGTH_DIFF
    )
    
    menor_fitness_observado_nesta_geracao = min(fitness_atualizado)

    if menor_fitness_observado_nesta_geracao < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado_nesta_geracao
        indice_melhor = fitness_atualizado.index(menor_fitness_geral)
        melhor_candidato_historico = populacao[indice_melhor]
        print(
            f"Geração: {geracao}, Melhor Fitness: {menor_fitness_geral}, Candidato: {''.join(melhor_candidato_historico)} (len: {len(melhor_candidato_historico)})"
        )

Geração: 1, Melhor Fitness: 296, Candidato: (Y@gj8mU0F# (len: 11)
Geração: 2, Melhor Fitness: 238, Candidato: OY@^Kf;=+-h# (len: 12)
Geração: 3, Melhor Fitness: 198, Candidato: OY@gjf;=+-h# (len: 12)
Geração: 4, Melhor Fitness: 176, Candidato: OY@gjf;U0-## (len: 12)
Geração: 5, Melhor Fitness: 151, Candidato: Q}@gjniU0>#% (len: 12)
Geração: 6, Melhor Fitness: 141, Candidato: (Y@gj_d50-9# (len: 12)
Geração: 7, Melhor Fitness: 96, Candidato: OYj^vfd56-9# (len: 12)
Geração: 8, Melhor Fitness: 84, Candidato: OYj^jfm06-## (len: 12)
Geração: 9, Melhor Fitness: 78, Candidato: OYjgjfi;0-#% (len: 12)
Geração: 10, Melhor Fitness: 77, Candidato: M}jglfm56&-% (len: 12)
Geração: 12, Melhor Fitness: 70, Candidato: MYiglfi50&-# (len: 12)
Geração: 13, Melhor Fitness: 66, Candidato: OY\gjfm50-## (len: 12)
Geração: 14, Melhor Fitness: 64, Candidato: OYiglfi0-,-# (len: 12)
Geração: 15, Melhor Fitness: 58, Candidato: MYiglfm50.-# (len: 12)
Geração: 16, Melhor Fitness: 49, Candidato: Oniglfm50,-# (len: 12)

Ótimo! Nossa algoritmo genético conseguiu descobrir a senha correta! Vamos exibir a senha encontrada e o número de gerações necessárias para encontrá-la.

In [7]:
print(f"\nSenha encontrada na geração {geracao}!")
print(f"Senha: {''.join(melhor_candidato_historico)}")


Senha encontrada na geração 243!
Senha: Thalles2001*


Nós precisamos de 243 gerações para encontrar a senha correta. Isso mostra que o algoritmo genético é eficiente para resolver problemas de otimização, mesmo quando a solução não é trivial. O ponto chave para derrotar essa fera formidável foi o fato da população inicial ser gerada com senhas de tamanhos variados, o que permitiu que o algoritmo explorasse diferentes combinações de caracteres e tamanhos. E, para a convergência, a mutação que altera o tamanho da senha foi crucial, pois permitiu que o algoritmo explorasse novas possibilidades de tamanho, aumentando as chances de encontrar a senha correta.